Skeleton Code

In [25]:
import requests
from bs4 import BeautifulSoup
import zipfile
import io
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
import time

# Function to download and load dataset
def load_data():
    page_url = 'https://archive.ics.uci.edu/dataset/240/human+activity+recognition+using+smartphones'
    page_response = requests.get(page_url)
    if page_response.status_code == 200:
        soup = BeautifulSoup(page_response.content, 'html.parser')
        download_link = soup.select_one('a[href$=".zip"]')['href']
        full_download_url = 'https://archive.ics.uci.edu' + download_link
        response = requests.get(full_download_url)
        if response.status_code == 200:
            with zipfile.ZipFile(io.BytesIO(response.content)) as outer_zip:
                inner_zip_name = 'UCI HAR Dataset.zip'
                with outer_zip.open(inner_zip_name) as inner_zip_file:
                    with zipfile.ZipFile(io.BytesIO(inner_zip_file.read())) as inner_zip:
                        with inner_zip.open('UCI HAR Dataset/train/X_train.txt') as myfile:
                            df = pd.read_csv(myfile, delim_whitespace=True, header=None)
                        with inner_zip.open('UCI HAR Dataset/train/y_train.txt') as myfile_y:
                            y = pd.read_csv(myfile_y, delim_whitespace=True, header=None)
    else:
        raise Exception("Failed to download or parse the dataset.")
    return df, y

In [26]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans
import numpy as np
import time

# Load dataset
df, y = load_data()
#TASK 1 - DO EDA and understand a little about the data.
#Only important thing is to know that it has a lot of features that don't make sense, just a
#bunch of readings from sensors.
#We think many of these features are redundant or irrelevant, and we want to find good features.
print("shape of dataset",df.shape)
print("first a five row",df.head())
print("type of features",df.dtypes)
print("missing value",df.isna().sum())
print("The description of data",df.describe())
for column in df.columns:
    print(f"Unique values in column {column}: {df[column].unique()}")

<ipython-input-25-836ccc4d8ebc>:30: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(myfile, delim_whitespace=True, header=None)
<ipython-input-25-836ccc4d8ebc>:32: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  y = pd.read_csv(myfile_y, delim_whitespace=True, header=None)


shape of dataset (7352, 561)
first a five row         0         1         2         3         4         5         6    \
0  0.288585 -0.020294 -0.132905 -0.995279 -0.983111 -0.913526 -0.995112   
1  0.278419 -0.016411 -0.123520 -0.998245 -0.975300 -0.960322 -0.998807   
2  0.279653 -0.019467 -0.113462 -0.995380 -0.967187 -0.978944 -0.996520   
3  0.279174 -0.026201 -0.123283 -0.996091 -0.983403 -0.990675 -0.997099   
4  0.276629 -0.016570 -0.115362 -0.998139 -0.980817 -0.990482 -0.998321   

        7         8         9    ...       551       552       553       554  \
0 -0.983185 -0.923527 -0.934724  ... -0.074323 -0.298676 -0.710304 -0.112754   
1 -0.974914 -0.957686 -0.943068  ...  0.158075 -0.595051 -0.861499  0.053477   
2 -0.963668 -0.977469 -0.938692  ...  0.414503 -0.390748 -0.760104 -0.118559   
3 -0.982750 -0.989302 -0.938692  ...  0.404573 -0.117290 -0.482845 -0.036788   
4 -0.979672 -0.990441 -0.942469  ...  0.087753 -0.351471 -0.699205  0.123320   

        555       556 

In [27]:
# Task 2: Encode class labels
# YOUR CODE HERE: Use LabelEncoder to encode class labels

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y.values.ravel()) # Flatten y to 1D array
encoded_y = pd.DataFrame(y_encoded)
print("Encoded class labels:", encoded_y)

Encoded class labels:       0
0     4
1     4
2     4
3     4
4     4
...  ..
7347  1
7348  1
7349  1
7350  1
7351  1

[7352 rows x 1 columns]


In [28]:
# Task 3: Scale the features using StandardScaler
# YOUR CODE HERE: Apply StandardScaler to df

scaler = StandardScaler()
df_scaled = scaler.fit_transform(df)
print("Scaled features:")
print(df_scaled)

Scaled features:
[[ 0.20064157 -0.0636826  -0.41962845 ... -0.68721921  0.40794614
  -0.00756789]
 [ 0.05594788  0.03148567 -0.25390836 ... -0.694138    0.40911698
   0.00787517]
 [ 0.07351535 -0.04341648 -0.07629468 ... -0.702239    0.4102883
   0.02650234]
 ...
 [-0.01566765  0.0167814   1.13222107 ... -0.56584847  0.64059683
   0.34870928]
 [ 0.21586648 -0.02812252 -0.86770988 ... -0.57766781  0.63147758
   0.29327564]
 [ 1.09620157  0.12919873 -1.67268082 ... -0.57392691  0.63274259
   0.33396081]]


In [29]:
# Task 4: Split the data into training and testing sets
# YOUR CODE HERE: Use train_test_split to split the data
X_train_full, X_test_full, y_train, y_test = train_test_split(df_scaled, y_encoded, test_size=0.2, random_state=42)
print("X_train_full:", X_train_full)
print("X_test_full:", X_test_full)
print("y_train:", y_train)
print("y_test:", y_test)

X_train_full: [[ 0.05007953 -0.00264025  0.00311082 ...  1.73598452 -2.03493103
  -1.34246107]
 [-0.52481104  0.41446321 -0.09420438 ...  0.03035389  1.09113703
   1.05869047]
 [ 0.23393768 -0.81315335  0.62903416 ... -0.64375157  0.54782554
   0.18969252]
 ...
 [ 0.27695402 -0.01582424 -0.1902678  ... -0.71704822  0.42007174
   0.07730518]
 [ 0.08520992 -0.03138312 -0.08149993 ... -0.6841837   0.07663221
  -0.14960606]
 [-0.15523612  0.60340855  0.24405698 ... -0.73967642  0.43579975
   0.21231841]]
X_test_full: [[ 1.24039460e-01 -7.18697560e-01 -1.05674427e+00 ... -5.76004317e-01
   5.52568937e-01 -3.57976041e-02]
 [-2.50278784e-01 -4.63824668e-01 -4.36416682e-01 ... -3.43139551e-01
   4.04700554e-01  1.07442981e+00]
 [ 2.39525340e-01  4.07524335e-01  6.47691200e-01 ... -7.18395750e-01
   4.51611834e-01  3.69864868e-01]
 ...
 [ 1.43586248e-01 -7.95462803e-02  3.98945231e-02 ...  3.25768990e-01
  -1.08288666e+00 -7.50821152e-01]
 [ 4.96957911e-02 -5.20606182e-02 -4.29794710e-04 ... -8

In [47]:
#  TASK 5 and 6

# Define a Pipeline with Gaussian Naive Bayes and train it on the full dataset
start_time = time.time()
classifier_pipeline_full = Pipeline([
    ('classifier', GaussianNB())
])
classifier_pipeline_full.fit(X_train_full, y_train)
y_pred_full = classifier_pipeline_full.predict(X_test_full)
end_time = time.time()
full_features_time = end_time - start_time
accuracy_full = accuracy_score(y_test, y_pred_full)

print("Accuracy with Full Features:", accuracy_full)
print("Training Time with Full Features:", full_features_time)
print(time.time())

Accuracy with Full Features: 0.7314751869476547
Training Time with Full Features: 0.11788702011108398
1736588845.0620766


In [44]:
# TASK 7 - K-Means for dimensionality reduction
n_clusters = 50  # Number of clusters for KMeans
kmeans = KMeans(n_clusters=n_clusters, random_state=42)  # Initialize KMeans object
kmeans.fit(df_scaled.T)  # Fit KMeans to the transposed data (features as data points)

# Get the cluster centers
cluster_centers = kmeans.cluster_centers_

# Calculate the distances between each feature and each cluster center
distances = np.linalg.norm(df_scaled.T[:, :, np.newaxis] - cluster_centers.T, axis=1)

# Get the index of the closest cluster for each feature
closest_clusters = np.argmin(distances, axis=1)

print("Closest Clusters for Each Feature:")
print(closest_clusters)
# Select features with the minimum distance to each cluster center
selected_features_indices = []
for cluster_id in range(n_clusters):
    # Find features closest to this cluster
    features_in_cluster = np.where(closest_clusters == cluster_id)[0]

    # If any features belong to this cluster, select the one closest to the center
    if len(features_in_cluster) > 0:
        closest_feature_in_cluster = features_in_cluster[np.argmin(distances[features_in_cluster, cluster_id])]
        selected_features_indices.append(closest_feature_in_cluster)

# Select the features from the original dataframe
selected_features = df.iloc[:, selected_features_indices]

print("Selected Features:")
print(selected_features)

Closest Clusters for Each Feature:
[42 42 37  9  1 10  9  1 10  9  1 10  2  2  2  9 18  1 10  9  1 10  4  4
  4  2  4 29 21 27 47 30 47 27  4 27 31  2 41 20 20 25 12 37  8  8 37  8
  8 20 25 12 20 25 12 25 20 25 12 37  8  8  7 25  8 14  7 14  7 21 31 21
 31 45 31 45 31 48 41 20 35 46 15  9  1  3  9  1  3  9  1  3  2  2  2  9
 18  1 22  9  1  3  4  4  4  2 14 29  2 27 47 30 30 27 21 27 45  2 41 20
 48 15 37 23 23 10 23 23 10 23  3  9  2  2  2 23 23 23 10  1 23 10 48  4
  4 27  4 36  2 17 34 17 34 27  4 27 44 20 39  4 35 42 32  1  3  9  1  3
  9  1  3  9  2  2  2  3 13 16 24  1  3  9  4  4  4 27  4 36 49 17 34 17
 17 27  4 27 27 20 39 26  9  9  9  9  4  9  9  9  4 45  0 27  0  9  9  9
  9  4  9  9  9  4 45  0 27  0  9  9  9  9  9  9 18  9  4 27  4 29 47 23
 23 23 23 10 23 23 23  4 21 31 21 31  3  3  3  3  1  3 16  3  4 27 34 17
 21  9  1  3  9  1 10  9  1 10  9  1 10 18 33 40  9 18  1 10  9  1  3  9
  4  4 14 21 21 14 45 45  7  7  6  6  6  6 18 18 18 18 18 18 18 18 18 18
 18 18 18 18  1 

In [48]:
#TASK 8 - Train another model (GaussianNB) on the new dataset, and report time taken and accuracy
from sklearn.naive_bayes import GaussianNB

# Split the selected features data into training and testing sets
X_train_selected, X_test_selected, y_train, y_test = train_test_split(
    selected_features, y_encoded, test_size=0.2, random_state=42
)

# Define a Pipeline with Gaussian Naive Bayes and train it on the selected features dataset
start_time = time.time()
classifier_pipeline_selected = Pipeline([('classifier', GaussianNB())])
classifier_pipeline_selected.fit(X_train_selected, y_train)
y_pred_selected = classifier_pipeline_selected.predict(X_test_selected)
end_time = time.time()
selected_features_time = end_time - start_time

# Calculate accuracy with selected features
accuracy_selected = accuracy_score(y_test, y_pred_selected)

print("Accuracy with Selected Features:", accuracy_selected)
print("Training Time with Selected Features:", selected_features_time)
print(time.time())

Accuracy with Selected Features: 0.822569680489463
Training Time with Selected Features: 0.016017913818359375
1736588852.9334064
